In [ ]:
import cv2
import torch
import numpy as np
from ultralytics import YOLO

# Load the custom-trained YOLOv5 model
model = YOLO('yolov8n.pt')

# Initialize the webcam
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("Error: Could not open video stream.")
    exit()
else:
    print("Webcam successfully opened!")

while True:
    ret, frame = cap.read()
    if not ret:
        print("Error: Failed to grab frame.")
        break

    # Run detection
    results = model(frame)

    # Annotate the frame with bounding boxes and labels using the plot() method
    frame = results[0].plot()  # This will display the bounding boxes directly on the frame

    # Display the annotated frame
    cv2.imshow("Fruit Detection", frame)

    # Exit when 'ESC' is pressed
    if cv2.waitKey(1) == 27:  # ESC key to exit
        break

# Release the webcam and close windows
cap.release()
cv2.destroyAllWindows()


In [12]:
import cv2
from ultralytics import YOLO



# Define your models
"""
models = [
    "runs/detect/cars_0_scratch/weights/best.pt",
]
"""
videos = ["vid4.mp4","vid5.mp4" ]#, "vid3.mp4", "vid4.mp4", "vid5.mp4"]
for n, video in enumerate(videos):
#if(True):
    #n = 4
    print(f"Video {n+1}:")
    models = [ [
        "runs/detect/cars_1_scratch/weights/best.pt",
        "runs/detect/cars_0_1_transfer_freeze/weights/best.pt"
        ],

        ["runs/detect/cars_0_scratch/weights/best.pt",
        "runs/detect/cars_1_scratch/weights/best.pt",
    "runs/detect/cars_0_1_transfer/weights/best.pt",
    "runs/detect/cars_0_1_transfer_freeze/weights/best.pt"
        ] ]
    
    crossing_line = [[300, 950, 950, 450, 550],[450, 975, 975, 575 , 675], [600, 1000, 1000, 700, 800]]

    # Line y-coordinate
    crossing_line_y = crossing_line[0][n+3]
    # Open the video
    cap = cv2.VideoCapture(video)
    assert cap.isOpened(), "Error reading video file"

    # Get video properties
    w, h, fps = (int(cap.get(x)) for x in (cv2.CAP_PROP_FRAME_WIDTH, cv2.CAP_PROP_FRAME_HEIGHT, cv2.CAP_PROP_FPS))

    # Initialize video writer (output video file)
    video_writer = cv2.VideoWriter("output_video.mp4", cv2.VideoWriter_fourcc(*"mp4v"), fps, (w, h))

    # Loop through each model and evaluate
    for model_path in models[0]:
        model = YOLO(model_path)
        cap.set(cv2.CAP_PROP_POS_FRAMES, 0)  # rewind video
        crossed_ids = set()
        car_count = 0  # Counter for cars crossing the line

        while True:
            ret, frame = cap.read()
            if not ret:
                break

            # Resize the frame to a smaller resolution
            resize_width = 1920  # Target width
            resize_height = 1080  # Target height
            frame = cv2.resize(frame, (resize_width, resize_height))
            results = model.track(frame, persist=True, conf=0.7, iou=0.5, tracker="bytetrack.yaml", verbose=False)

            # if results[0].boxes.id is None:
            #     continue
            annotated_frame = frame
            if results[0].boxes.id is not None:
                ids = results[0].boxes.id.cpu().tolist()
                boxes = results[0].boxes.xyxy.cpu().tolist()
                annotated_frame = results[0].plot()
                

                for track_id, (x1, y1, x2, y2) in zip(ids, boxes):
                    cx = (x1 + x2) / 2
                    cy = (y1 + y2) / 2

                    if track_id not in crossed_ids and cy < crossing_line_y:
                        crossed_ids.add(track_id)
                        car_count += 1  # Increment car count
                        #print(f"Model {model_path}: Track ID {track_id} crossed.")
                    cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)

            # Draw the crossing line
            cv2.line(annotated_frame, (0, crossing_line_y), (annotated_frame.shape[1], crossing_line_y), (0, 255, 255), 2)
            # Display car count on frame
            cv2.putText(annotated_frame, f'Cars Crossed: {car_count}', (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

            # Show frame with line and car count
            cv2.imshow(f"{model_path}", annotated_frame)

            # Write to video output
            video_writer.write(annotated_frame)

            # Slow down the video by increasing the wait time (delay between frames)
            if cv2.waitKey(5) == 27:  # Increase wait time from default 1 to 100ms
                cv2.destroyAllWindows()
                break

        print(f"\n✅ Model {model_path} - Total cars crossed line: {car_count}\n")
        cv2.destroyAllWindows()
    
# Release resources
cap.release()
video_writer.release()
cv2.destroyAllWindows()


Video 1:

✅ Model runs/detect/cars_1_scratch/weights/best.pt - Total cars crossed line: 23


✅ Model runs/detect/cars_0_1_transfer_freeze/weights/best.pt - Total cars crossed line: 19

Video 2:

✅ Model runs/detect/cars_1_scratch/weights/best.pt - Total cars crossed line: 22


✅ Model runs/detect/cars_0_1_transfer_freeze/weights/best.pt - Total cars crossed line: 21



In [60]:
#pre trained without fine tuning.
#model1 = YOLO('runs/detect/cars_0_scratch/weights/best.pt')  # load a pretrained model
model2 = YOLO('runs/detect/cars_1_scratch/weights/best.pt')
model3 = YOLO('runs/detect/cars_0_1_transfer/weights/best.pt')
model4 = YOLO('runs/detect/cars_0_1_transfer_freeze/weights/best.pt')

#result1 = model1.val(data='data/traffic.yaml', split = 'test')
result2 = model2.val(data='car_dataset/data.yaml', split = 'test') 
result3 = model3.val(data='car_dataset/data.yaml', split = 'test')  
result4 = model4.val(data='car_dataset/data.yaml', split = 'test') 
#print(f'pretrained model \nAccuracy: {result1.box.map:.2f}')
print(f'pretrained and fine tuned (20 epochs) \nAccuracy: {result2.box.map:.2f}')
print(f'Scratch (5 epochs) \nAccuracy: {result3.box.map:.2f}')  
print(f'Scratch (15 epochs) \nAccuracy: {result4.box.map:.2f}')  


Ultralytics 8.3.102  Python-3.11.5 torch-2.5.1+cu124 CUDA:0 (NVIDIA GeForce RTX 4070, 12282MiB)
Model summary (fused): 72 layers, 11,125,971 parameters, 0 gradients, 28.4 GFLOPs


val: Scanning D:\VsCode\Computer_Vision\CNN.YOLO\car_dataset\labels\test.cache... 49 images, 0 backgrounds, 0 corrupt: 100%|██████████| 49/49 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:03<00:00,  1.23it/s]


                   all         49        317      0.828      0.902      0.924      0.769
Speed: 0.5ms preprocess, 3.5ms inference, 0.0ms loss, 1.6ms postprocess per image
Results saved to runs\detect\val
Ultralytics 8.3.102  Python-3.11.5 torch-2.5.1+cu124 CUDA:0 (NVIDIA GeForce RTX 4070, 12282MiB)
Model summary (fused): 72 layers, 11,125,971 parameters, 0 gradients, 28.4 GFLOPs


val: Scanning D:\VsCode\Computer_Vision\CNN.YOLO\car_dataset\labels\test.cache... 49 images, 0 backgrounds, 0 corrupt: 100%|██████████| 49/49 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:03<00:00,  1.32it/s]


                   all         49        317      0.827      0.849       0.91      0.762
Speed: 1.1ms preprocess, 5.9ms inference, 0.0ms loss, 1.7ms postprocess per image
Results saved to runs\detect\val2
Ultralytics 8.3.102  Python-3.11.5 torch-2.5.1+cu124 CUDA:0 (NVIDIA GeForce RTX 4070, 12282MiB)
Model summary (fused): 72 layers, 11,125,971 parameters, 0 gradients, 28.4 GFLOPs


val: Scanning D:\VsCode\Computer_Vision\CNN.YOLO\car_dataset\labels\test.cache... 49 images, 0 backgrounds, 0 corrupt: 100%|██████████| 49/49 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:03<00:00,  1.31it/s]


                   all         49        317      0.821      0.849      0.891      0.724
Speed: 0.6ms preprocess, 7.2ms inference, 0.0ms loss, 1.5ms postprocess per image
Results saved to runs\detect\val3
pretrained and fine tuned (20 epochs) 
Accuracy: 0.77
Scratch (5 epochs) 
Accuracy: 0.76
Scratch (15 epochs) 
Accuracy: 0.72


<Figure size 1000x600 with 1 Axes>

<Figure size 1000x600 with 1 Axes>

<Figure size 1000x600 with 1 Axes>